<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U12/ANN12_Denoising_Autoencoder_TEMPLATE_pl.ipynb)

# Denoising Autoencoder für MNIST-Ziffern


## Einführung

Dieses Beispiel zeigt, wie man einen tiefen Faltungs-Autoencoder für die Bildentrauschung implementiert, um aus verrauschten Ziffernbilder aus dem MNIST-Datensatz saubere Ziffernbilder zu erzeugen. Diese Implementierung basiert auf einem ursprünglichen Blogbeitrag mit dem Titel [Building Autoencoders in Keras](https://blog.keras.io/building-autoencoders-in-keras.html)
von [François Chollet](https://twitter.com/fchollet).


In [ ]:
import time
import torch
import webbrowser
import subprocess
import numpy as np
import torch.nn as nn
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import torch.nn.functional as F

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


## (a) Setup, Hilfsfunktionen für das Preprocessing
### Hilfsfunktionen zur Datenvorbereitung, Visualisierung und Evaluation

Dieser Abschnitt enthält unterstützende Funktionen zur Vorbereitung und Auswertung von Bilddaten, insbesondere für Denoising Autoencoder auf dem MNIST-Datensatz.

##### `preprocess(array)`
- **Zweck**: Normalisiert die Pixelwerte von Bildern auf den Bereich `[0, 1]` und formt sie in das Format `(Batch, Höhe, Breite, Kanäle)` um.
- **Verwendung**: Wird auf Rohbilder angewendet, um sie für ein neuronales Netz vorzubereiten.
- **Hinweis**: Das Kanalformat ist auf 1 (graustufige Bilder) gesetzt.

##### `noise(array, noise_factor=0.4)`
- **Zweck**: Fügt Gaußsches Rauschen zu den Bildern hinzu, um verrauschte Eingabedaten für den Denoising Autoencoder zu generieren.
- **Parameter**:
  - `noise_factor`: Stärke des hinzugefügten Rauschens.
- **Rückgabe**: Bildarray mit Rauschen, begrenzt auf Werte zwischen `0.0` und `1.0`.

##### `display_images(original, compared)`
- **Zweck**: Visualisiert zufällig ausgewählte Originalbilder und die dazugehörigen Vergleichsbilder (z. B. verrauschte oder rekonstruierte Bilder).
- **Darstellung**: Zeigt 10 Bildpaare in zwei Reihen – oben die Originalbilder, unten die Vergleichsbilder.

##### `display_multiple_predictions(model, dataset, n_blocks=5, n_images=10)`
- **Zweck**: Führt mehrere Vorhersageblöcke mit dem Denoising-Modell aus und zeigt die Ergebnisse.
- **Ablauf**:
  - Wählt zufällige Beispiele aus dem Dataset aus.
  - Führt Vorwärtsdurchlauf durch das Modell aus.
  - Visualisiert jeweils `n_images` verrauschte und rekonstruierte Bilder in mehreren Blöcken (`n_blocks`).
- **Voraussetzung**: Das Dataset muss Paare `(noisy_image, clean_image)` zurückgeben.

In [ ]:
# ------------------------
# (a) Hilfsfunktionen
# ------------------------
# Normalisierung und Umformung der Bilder
def preprocess(array):
    array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), 28, 28, 1))
    return array


# Hinzufügen von Rauschen zum Bild (Gaussian Noise)
def noise(array, noise_factor=0.4):
    noisy_array = array + noise_factor * np.random.normal(0.0, 1.0, array.shape)
    return np.clip(noisy_array, 0.0, 1.0)


# Darstellung von Original- und Vergleichsbildern (z. B. noisy vs. recon)
def display_images(original, compared):
    n = 10
    indices = np.random.randint(len(original), size=n)
    original, compared = original[indices], compared[indices]

    fig = plt.figure(figsize=(20, 4))
    fig.suptitle("Original vs. Vergleichsbilder", fontsize=16)
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(original[i].reshape(28, 28), cmap="gray")
        ax.axis("off")
        ax.set_title("Original", fontsize=10)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(compared[i].reshape(28, 28), cmap="gray")
        ax.axis("off")
        ax.set_title("Vergleich", fontsize=10)
    plt.show()


# Darstellung mehrerer zufälliger Vorhersageblöcke
def display_multiple_predictions(model, dataset, n_blocks=5, n_images=10):
    model.eval()
    for block_idx in range(n_blocks):
        idxs = np.random.choice(len(dataset), n_images, replace=False)
        noisy = torch.stack([dataset[i][0] for i in idxs])
        clean = torch.stack([dataset[i][1] for i in idxs])

        with torch.no_grad():
            denoised = model(noisy)

        fig = plt.figure(figsize=(20, 4))
        fig.suptitle(f"Vorhersageblock {block_idx + 1}", fontsize=16)
        display_images(
            noisy.numpy().transpose(0, 2, 3, 1), denoised.numpy().transpose(0, 2, 3, 1)
        )


## (b) Datenvorbereitung

In diesem Abschnitt wird ein spezielles Dataset für den Denoising Autoencoder erstellt. Dabei werden verrauschte Versionen der MNIST-Bilder generiert und gemeinsam mit den sauberen Bildern verwendet.

#### `NoisyMNIST` Dataset-Klasse
- **Zweck**: Erstellt ein eigenes `Dataset`-Objekt mit verrauschten Eingabebildern und den dazugehörigen sauberen Zielbildern.
- **Details**:
  - Die MNIST-Daten werden geladen und als `numpy`-Array verarbeitet.
  - Die Bilder werden mit der Funktion `preprocess()` normalisiert und umgeformt.
  - Über `noise()` wird Gaußsches Rauschen hinzugefügt.
- **`__getitem__`**:
  - Gibt jeweils ein Paar `(noisy_image, clean_image)` im Format `(Kanäle, Höhe, Breite)` zurück – konvertiert zu PyTorch-Tensoren.

#### DataLoader
```python
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128)
```
- Lädt die Daten in Batches von 128 Bildern.
- Beim Training werden die Daten zufällig durchmischt (`shuffle=True`).

#### Beispielbilder anzeigen
```python
examples = next(iter(train_loader))
display_images(
    examples[1].numpy().transpose(0, 2, 3, 1),  # Saubere Bilder (Ziel)
    examples[0].numpy().transpose(0, 2, 3, 1),  # Verrauschte Bilder (Eingabe)
)
```
- Zeigt 10 zufällig ausgewählte Bildpaare: **sauber (Original)** vs. **verrauscht (Input)**.
- Die Darstellung dient der Überprüfung, ob das Rauschen korrekt hinzugefügt wurde und ob die Datenstruktur passt.

In [ ]:
# ------------------------
# (b) Datenvorbereitung
# ------------------------
# Dataset mit verrauschten und sauberen Bildern
class NoisyMNIST(Dataset):
    def __init__(self, train=True):
        data = MNIST("data", train=train, download=True).data.numpy()
        self.clean = preprocess(data)
        self.noisy = noise(self.clean)

    def __len__(self):
        return len(self.clean)

    def __getitem__(self, idx):
        x = torch.tensor(self.noisy[idx], dtype=torch.float32).permute(2, 0, 1)
        y = torch.tensor(self.clean[idx], dtype=torch.float32).permute(2, 0, 1)
        return x, y


# Laden der Trainings- und Testdaten
train_dataset = NoisyMNIST(train=True)
test_dataset = NoisyMNIST(train=False)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128)

# Beispielbilder darstellen (clean vs. noisy)
examples = next(iter(train_loader))
display_images(
    examples[1].numpy().transpose(0, 2, 3, 1), examples[0].numpy().transpose(0, 2, 3, 1)
)


## (c) Aufbau des Autoencoders
In diesem Abschnitt wird ein einfacher **Convolutional Autoencoder** mit PyTorch Lightning implementiert. Das Modell besteht aus einem **Encoder** zum Komprimieren und einem **Decoder** zum Rekonstruieren der Eingabebilder.

#### `LitAutoEncoder` (LightningModule)
Ein vollständig trainierbares Autoencoder-Modell mit Trainings- und Validierungsschritten.

#### Encoder
```python
self.encoder = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 32, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
)
```
- **Ziel**: Komprimiert das Eingabebild durch zwei Faltungs- und Pooling-Stufen.
- **Details**:
  - `Conv2d`: Extrahiert Merkmale aus dem Bild.
  - `MaxPool2d`: Halbiert jeweils die räumliche Auflösung.

#### Decoder
```python
self.decoder = nn.Sequential(
    nn.ConvTranspose2d(32, 32, 3, stride=2, padding=1, output_padding=1),
    nn.ReLU(),
    nn.ConvTranspose2d(32, 32, 3, stride=2, padding=1, output_padding=1),
    nn.ReLU(),
    nn.Conv2d(32, 1, 3, padding=1),
    nn.Sigmoid(),
)
```
- **Ziel**: Rekonstruiert das Bild schrittweise aus der komprimierten Darstellung.
- **Details**:
  - `ConvTranspose2d`: Führt Upsampling durch (umgekehrte Faltung).
  - `Sigmoid`: Begrenzt die Ausgabewerte auf `[0, 1]` zur Bildrekonstruktion.

#### `forward(x)`
- Führt die Eingabe durch Encoder und Decoder.
- Gibt das rekonstruierte Bild zurück.

#### `training_step`
```python
loss = F.binary_cross_entropy(self(x), y)
```
- Führt einen Trainingsschritt mit **Binary Cross Entropy** zwischen Rekonstruktion und Zielbild durch.
- Loggt den Trainingsverlust als `"train_loss"`.

#### `validation_step`
- Führt einen Validierungsschritt identisch zum Training aus.
- Loggt den Validierungsverlust als `"val_loss"`.

#### `configure_optimizers`
```python
return torch.optim.Adam(self.parameters(), lr=1e-3)
```
- Verwendet den **Adam-Optimierer** mit Lernrate `1e-3` zur Optimierung aller Modellparameter.

In [ ]:
# ------------------------
# (c) Autoencoder-Modell
# ------------------------
# Einfacher Conv-Autoencoder mit 2 Downsampling- und 2 Upsampling-Stufen
class LitAutoEncoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
       #insert your code here


    def forward(self, x):
       #insert your code here

    def training_step(self, batch, batch_idx):
        #insert your code here

    def validation_step(self, batch, batch_idx):
        #insert your code here
        
    def configure_optimizers(self):
        #insert your code here

## (d) Training des Autoencoders
In diesem Abschnitt wird das Autoencoder-Modell mit PyTorch Lightning trainiert. Dabei kommen **Early Stopping** und ein **TensorBoard Logger** zum Einsatz.

#### Modellinitialisierung
```python
model = LitAutoEncoder()
```
- Erstellt eine Instanz des zuvor definierten Autoencoder-Modells.

#### TensorBoard Logger
```python
logger = TensorBoardLogger("tb_logs", name="denoising_ae")
```
- Speichert Trainingsmetriken zur Visualisierung mit TensorBoard.
- Logs werden im Verzeichnis `tb_logs/denoising_ae/` abgelegt.

#### Trainer-Konfiguration
```python
trainer = pl.Trainer(
    max_epochs=20,
    logger=logger,
    accelerator="auto",
    devices=1,
    callbacks=[EarlyStopping(monitor="train_loss", patience=5)],
)
```
- **`max_epochs=20`**: Training wird maximal 20 Epochen lang durchgeführt.
- **`accelerator="auto"`**: Automatische Auswahl von CPU oder GPU.
- **`devices=1`**: Nutzt ein einzelnes Gerät (GPU oder CPU).
- **`EarlyStopping`**: Bricht das Training frühzeitig ab, wenn sich der Trainingsverlust (`train_loss`) 5 Epochen lang nicht verbessert.

#### Training starten
```python
trainer.fit(model, train_loader, test_loader)
```
- Startet das Training mit den definierten Trainings- und Test-Daten.
- Das Test-Set wird hier als Validierungs-Set verwendet.

In [ ]:
# ------------------------
# (d) Training
# ------------------------
# Trainiere Modell mit EarlyStopping und TensorBoard-Logger
model = LitAutoEncoder()
logger = TensorBoardLogger("tb_logs", name="denoising_ae")
trainer = pl.Trainer(
    max_epochs=20,
    logger=logger,
    accelerator="auto",
    devices=1,
    callbacks=[EarlyStopping(monitor="train_loss", patience=5)],
)
trainer.fit(model, train_loader, test_loader)


## (e) Speichern des Modells
Nach dem Training wird das Modell gespeichert, um es später erneut laden und verwenden zu können – z. B. für Inferenz oder Fine-Tuning.

#### Speichern der Modellgewichte
```python
torch.save(model.state_dict(), "AE_weights.pt")
```
- Speichert die **gelernten Gewichte** des Modells in einer Datei namens `"AE_weights.pt"`.
- Nur die **Gewichte** werden gespeichert, nicht die gesamte Modellstruktur.
- Kann später mit `model.load_state_dict(torch.load("AE_weights.pt"))` wieder geladen werden.

In [ ]:
# ------------------------
# (e) Modell speichern
# ------------------------
# Speicher die gelernten Gewichte
torch.save(model.state_dict(), "AE_weights.pt")


## (f) Lernkurven anzeigen
In diesem Schritt wird **TensorBoard** gestartet, um das Training visuell auszuwerten – z. B. den Verlauf von Verlustfunktionen oder anderen Metriken.

####  Starten von TensorBoard
```python
logdir = "tb_logs/denoising_ae"
subprocess.Popen(["tensorboard", "--logdir", logdir])
```
- Startet einen TensorBoard-Server im Hintergrund.
- Verwendet das zuvor definierte Logverzeichnis `"tb_logs/denoising_ae"`.

#### Browser öffnen
```python
url = "http://localhost:6006/"
time.sleep(2)
webbrowser.open(url)
```
- Wartet 2 Sekunden, um sicherzustellen, dass TensorBoard gestartet ist.
- Öffnet anschließend automatisch den Browser unter `http://localhost:6006/`.

In [ ]:
# ------------------------
# (f) TensorBoard anzeigen
# ------------------------
# Starte TensorBoard und öffne Browser zur Visualisierung
logdir = "tb_logs/denoising_ae"
url = "http://localhost:6006/"
subprocess.Popen(["tensorboard", "--logdir", logdir])
time.sleep(2)
webbrowser.open(url)


## (g) Vorhersagen machen (Rekonstruktion der Testdaten)
In diesem letzten Schritt wird das trainierte Modell verwendet, um verrauschte **Testbilder** zu rekonstruieren. Anschließend werden Original vs. Rekonstruktion visualisiert.

#### Modell in Evaluierungsmodus versetzen
```python
model.eval()
```
- Deaktiviert Dropout und BatchNorm.
- Modell befindet sich jetzt im **Evaluierungsmodus** (wichtig für konsistente Vorhersagen).

#### Vorhersage-Schleife über Testdaten
```python
preds = []
for batch in test_loader:
    x, _ = batch
    with torch.no_grad():
        preds.append(model(x))
```
- Geht alle Batches im Test-Loader durch.
- Führt Vorhersagen im **no_grad**-Kontext aus (kein Gradiententracking → effizienter).
- Ergebnisse werden in der Liste `preds` gesammelt.

#### Zusammensetzen & Umwandeln der Vorhersagen
```python
pred_images = torch.cat(preds, dim=0).numpy().transpose(0, 2, 3, 1)
```
- Alle Batches werden zu einem großen Array zusammengesetzt.
- Anschließend wird die Tensorform von `(Batch, Channels, Height, Width)` zu `(Batch, Height, Width, Channels)` konvertiert – für die Bildanzeige.

#### Visualisierung: Noisy vs. Reconstructed
```python
display_images(test_dataset.noisy, pred_images)
```
- Zeigt eine Auswahl an verrauschten Testbildern (Input) und den rekonstruierten Bildern (Output des Autoencoders).
- Ideal zur qualitativen Bewertung der Modellleistung.

In [ ]:
# ------------------------
# (g) Test-Vorhersage
# ------------------------
# Rekonstruiere verrauschte Testbilder
model.eval()
preds = []
for batch in test_loader:
    x, _ = batch
    with torch.no_grad():
        preds.append(model(x))

pred_images = torch.cat(preds, dim=0).numpy().transpose(0, 2, 3, 1)
display_images(test_dataset.noisy, pred_images)


## (h) Denoising mit dem Autoencoder (100 Epochen)


In [ ]:
# ------------------------
# (h) Training mit 100 Epochen
# ------------------------
# Optional: längeres Training zur Verbesserung der Rekonstruktion
model = LitAutoEncoder()
trainer = pl.Trainer(
    max_epochs=100,
    logger=logger,
    accelerator="auto",
    devices=1,
    callbacks=[EarlyStopping(monitor="train_loss", patience=5)],
)
trainer.fit(model, train_loader, test_loader)


## (i) Rauschfiltertest (Denoising mit Testdaten)
Dieser Schritt ist **optional** und dient dazu, die Qualität der Rekonstruktionen weiter zu verbessern, indem das Modell **länger trainiert** wird.

#### Neues Modell initialisieren
```python
model = LitAutoEncoder()
```
- Erstellt ein neues Autoencoder-Modell.
- Hinweis: Wenn du das bereits trainierte Modell weitertrainieren möchtest, kannst du alternativ die gespeicherten Gewichte laden.

#### Trainer mit 100 Epochen konfigurieren
```python
trainer = pl.Trainer(
    max_epochs=100,
    logger=logger,
    accelerator="auto",
    devices=1,
    callbacks=[EarlyStopping(monitor="train_loss", patience=5)],
)
```
- **`max_epochs=100`**: Maximale Trainingsdauer beträgt 100 Epochen.
- **`EarlyStopping`**: Training wird frühzeitig beendet, wenn sich der Trainingsverlust 5 Epochen lang nicht verbessert.
- **`logger`**: TensorBoard Logger wird wiederverwendet (aus vorherigem Schritt).

#### Training starten
```python
trainer.fit(model, train_loader, test_loader)
```
- Startet das verlängerte Training mit den gleichen Trainings- und Testdaten.
- Ziel: Bessere Rekonstruktionen durch längeres Lernen.

In [ ]:
# ------------------------
# (i) Finaler Denoising-Test
# ------------------------
# Rekonstruiere komplette Testmenge und speichere das Bild
model.eval()
preds = []
for batch in test_loader:
    x, _ = batch
    with torch.no_grad():
        preds.append(model(x))

pred_images = torch.cat(preds, dim=0).numpy().transpose(0, 2, 3, 1)
display_images(test_dataset.noisy, pred_images)


## (j) Mehrfach-Vorhersagen anzeigen (5 Bildblöcke)
Dieser Schritt dient der **visuellen Evaluation des Modells** über mehrere zufällig ausgewählte Bildgruppen hinweg. So lassen sich unterschiedliche Rekonstruktionsbeispiele einfach vergleichen.

#### Mehrfachanzeige von Vorhersageblöcken
```python
display_multiple_predictions(model, test_dataset, n_blocks=5)
```
- **Zweck**: Zeigt 5 verschiedene Vorhersageblöcke mit jeweils 10 Bildern.
- Für jeden Block werden zufällig:
  - **10 verrauschte Bilder (Input)** aus dem Testset ausgewählt
  - die **rekonstruierten Bilder** mit dem Autoencoder erzeugt
- Jedes Bildpaar (oben: noisy, unten: reconstructed) wird zur qualitativen Einschätzung dargestellt.
- Besonders hilfreich, um die Modellleistung bei **verschiedenen Bildtypen** visuell zu analysieren.


In [ ]:
# ------------------------
# (j) Mehrfache Vorhersagen
# ------------------------
# Zeige 5 zufällige Blöcke an Vorhersagen
display_multiple_predictions(model, test_dataset, n_blocks=5)
